# ResNet50 Model On QuickDraw Dataset

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, applications, callbacks
from sklearn.model_selection import train_test_split

In [ ]:
# Configuration
NUM_CLASSES = 30
SAMPLES_PER_CLASS = 5000
IMG_SIZE = (28, 28)
BATCH_SIZE = 64  # Reduced for 16GB RAM
EPOCHS = 50
TEST_SIZE = 0.2

In [ ]:
# 1. Data Loading (from .npy files)
def load_data(class_names, samples_per_class=5000):
    X, y = [], []
    for class_idx, name in enumerate(class_names):
        data = np.load(f"{name.lower()}.npy")[:samples_per_class]
        X.append(data)
        y.append(np.full(len(data), class_idx))
    return np.concatenate(X), np.concatenate(y)

# List of classes
class_names = ['Airplane', 'Apple', 'Bicycle', 'Book', 'Car', 'Cat', 'Chair', 'Clock', 'Dog', 'Door', 'Eye', 'Fish', 'Flower', 'Fork', 'House', 'Key', 'Ladder', 'Moon', 'Mountain', 'Pizza', 'Rainbow', 'Shoe', 'Smiley Face', 'Star', 'Stop Sign', 'Sun', 'Table', 'Tennis Racquet', 'Tree', 'Wheel']
X, y = load_data(class_names)

# 2. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, stratify=y, random_state=42
)

# 3. Add channel dimension
X_train = X_train.reshape(-1, 28, 28, 1)
X_test  = X_test.reshape(-1, 28, 28, 1)


In [ ]:
# 4. Preprocessing
def preprocess(images):
    # Rescale to [0,1]
    return images / 255.0

X_train = preprocess(X_train)
X_test = preprocess(X_test)

# 5. Data Augmentation
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

In [ ]:

def build_resnet(input_shape=(28,28,1), num_classes=30):
    # Input layer
    inputs = layers.Input(shape=input_shape)
    
    # Convert grayscale to RGB by repeating channels
    x = layers.Concatenate()([inputs]*3)
    
    # Modified ResNet50 for smaller images
    base_model = applications.ResNet50(
        include_top=False,
        weights=None,  # No pretrained weights
        input_tensor=x,
        pooling='avg'
    )
    
    # Custom classification head
    x = layers.Dense(512, activation='relu')(base_model.output)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    return models.Model(inputs, outputs)

model = build_resnet()
model.summary()

# 6. Compile with Mixed Precision
tf.keras.mixed_precision.set_global_policy('mixed_float16')
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 7. Callbacks
callbacks_list = [
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_resnet.h5', save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)
]

# 8. Training with Data Generators
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=lambda x: data_augmentation(x, training=True)
).flow(X_train, y_train, BATCH_SIZE)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator().flow(X_test, y_test, BATCH_SIZE)

history = model.fit(
    train_gen,
    steps_per_epoch=len(X_train)//BATCH_SIZE,
    validation_data=test_gen,
    validation_steps=len(X_test)//BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks_list
)


In [ ]:
# 9. Evaluation
model.load_weights('best_resnet.h5')
test_loss, test_acc = model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc:.4f}")